In [8]:
import numpy as np
from scipy.optimize import least_squares

In [9]:
# columns of matrix obs are possible observations of
# binary nucleotides associated with the four species
#
# the set of possible observations is {-1, +1}^4

obs = np.array([[-1, -1, -1, -1, -1, -1, -1, -1, +1, +1, +1, +1, +1, +1, +1, +1], 
                [-1, -1, -1, -1, +1, +1, +1, +1, -1, -1, -1, -1, +1, +1, +1, +1], 
                [-1, -1, +1, +1, -1, -1, +1, +1, -1, -1, +1, +1, -1, -1, +1, +1], 
                [-1, +1, -1, +1, -1, +1, -1, +1, -1, +1, -1, +1, -1, +1, -1, +1]])

# adjust the following parameters before computing mean and covariance

global tau # \theta_{1} * \theta_{5} * \theta_{2}
tau = 0.5

global k # length of DNA sequence
k = 300

In [10]:
# assume our observations are k i.i.d X_1,...,X_k
# taking values from the columns of obs
# 
# also assume \theta_{3} = \theta_{4} = 0
# 
# q_j = P(X_i = obs[:, j]) i.e. q_j is the probability
# of observing column j of obs

q = np.zeros(16)
for j in range(16):
  q[j] = (1 / 16) * (1 + obs[0, j] * obs[1, j] * tau)

print(q)

[0.09375 0.09375 0.09375 0.09375 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.09375 0.09375 0.09375 0.09375]


In [11]:
# let Y_i = e_j \in \mathbb{R}^16 iff X_i = obs[:, j]
# where e_1,...,e_16 are the standard basis vectors, and
# define frequency vector F_k = Y_1 + ... + Y_k
#
# our frequency vector follows multinomial distribution with
# k trials and event probabilities q, and it can be approximated
# as a multivariate Gaussian via the Central Limit Theorem; see
# computation in Overleaf document
#
# see https://en.wikipedia.org/wiki/Multinomial_distribution#Properties
# and https://en.wikipedia.org/wiki/Central_limit_theorem#Multidimensional_CLT
#
# we now compute the mean and covariance

mean = k * q # mean vector of multivariate Gaussian

cov = k * (np.diag(q) - np.outer(q, q)) # cov matrix of multivariate Gaussian

print(mean, "\n")
print(cov)

# copy and paste as a np.array(...)

[28.125 28.125 28.125 28.125  9.375  9.375  9.375  9.375  9.375  9.375
  9.375  9.375 28.125 28.125 28.125 28.125] 

[[25.48828125 -2.63671875 -2.63671875 -2.63671875 -0.87890625 -0.87890625
  -0.87890625 -0.87890625 -0.87890625 -0.87890625 -0.87890625 -0.87890625
  -2.63671875 -2.63671875 -2.63671875 -2.63671875]
 [-2.63671875 25.48828125 -2.63671875 -2.63671875 -0.87890625 -0.87890625
  -0.87890625 -0.87890625 -0.87890625 -0.87890625 -0.87890625 -0.87890625
  -2.63671875 -2.63671875 -2.63671875 -2.63671875]
 [-2.63671875 -2.63671875 25.48828125 -2.63671875 -0.87890625 -0.87890625
  -0.87890625 -0.87890625 -0.87890625 -0.87890625 -0.87890625 -0.87890625
  -2.63671875 -2.63671875 -2.63671875 -2.63671875]
 [-2.63671875 -2.63671875 -2.63671875 25.48828125 -0.87890625 -0.87890625
  -0.87890625 -0.87890625 -0.87890625 -0.87890625 -0.87890625 -0.87890625
  -2.63671875 -2.63671875 -2.63671875 -2.63671875]
 [-0.87890625 -0.87890625 -0.87890625 -0.87890625  9.08203125 -0.29296875
  -0.29296875

In [5]:
# we can now generate data by sampling a point from this distribution
# by executing the following command

freq = np.random.multivariate_normal(mean, cov)
print(freq, "\n")

# gives frequency of observations, would need to round components
# in practice

[26.18595552 19.88178106 32.3619518  25.1850168  11.72939597  4.96815609
  8.61220286  7.62459227 11.13078244 16.30505883  8.51878733  7.4611995
 29.28236207 27.01672445 30.94814749 32.78788534] 



In [24]:
# no idea if this is right lol
# see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html
# and https://hernandis.me/2020/04/05/three-examples-of-nonlinear-least-squares-fitting-in-python-with-scipy.html

# return random vector z from frequency vector
def data(freq, q, k):
  return np.sqrt(k) * ((freq / k) - q) # is this correct?

# p = (\theta_1, \theta_2, \alpha_3, \alpha_4)
# q = [q_j] for j = 1,...,16 i.e. observation probabilities 
# tau is a known parameter
# z is our data
#
# returns vector of residuals for least squares
# see least squares problem in Overleaf document
def llhd(p, q, tau, z):
  epsilon = np.zeros(16)
  for j in range(16):
    # formula for /epsilon_{/sigma_j}
    # multiply by constraint(p, tau) to force 0 <= \theta_5 <= 1
    epsilon[j] = constraint(p, tau) * ((obs[0, j]*obs[2, j]*p[0]*p[2]) + (obs[1, j]*obs[3, j]*p[1]*p[3]) + (obs[0, j]*obs[3, j]*(tau/p[1])*p[3]) + obs[1, j]*obs[2, j]*(tau/p[0])*p[2])
  return np.multiply(1 / np.sqrt(q), z - epsilon)

# p = (\theta_1, \theta_2, \alpha_3, \alpha_4)
# tau is a known parameter
#
# nonlinear constraint for nonlinear least squares
# forces 0 <= \theta_5 <= 1 where
# \theta_1 * \theta_2 * \theta_5 = \tau
def constraint(p, tau):
  if tau/(p[0]*p[1]) < 0 or tau/(p[0]*p[1]) > 1:
    return 100 # np.inf
  return 1

freq = np.random.multivariate_normal(mean, cov)
z = data(freq, q, k)

# function passed to scipy.optimize.least_squares
def fun(p):
  return llhd(p, q, tau, z)
  
# minimizer of least sqaures
res = least_squares(fun, np.ones(4), bounds=(np.array([0, 0, 0, 0]), np.array([1, 1, np.inf, np.inf])))

epsilon = np.zeros(16)
for j in range(16):
    epsilon[j] = (obs[0, j]*obs[2, j]*res.x[0]*res.x[2]) + (obs[1, j]*obs[3, j]*res.x[1]*res.x[3]) + (obs[0, j]*obs[3, j]*(tau/res.x[1])*res.x[3]) + obs[1, j]*obs[2, j]*(tau/res.x[0])*res.x[2]

print("F_k: ", freq, "\n")
print("z_sigma: ", z, "\n")
print("e_sigma: ", epsilon, "\n")
print("Cost: ", res.cost, "\n")
# print(0.5 * np.sum(np.square(llhd(res.x, q, tau, z)))) # manual computation gives same minimum cost
print("theta_1", res.x[0])
print("theta_2", res.x[1])
print("alpha_3", res.x[2])
print("alpha_4", res.x[3])
print("theta_5", tau/(res.x[0]*res.x[1]))

F_k:  [30.80596269 24.58432557 39.87786997 32.50246293  9.91596675  8.38598576
 10.0997625   7.33285352  8.86728836  4.82404767  4.62937719 13.26463085
 21.5180444  27.07776757 24.15890255 32.15475216] 

z_sigma:  [ 0.15478545 -0.20442093  0.67855226  0.25273294  0.03123273 -0.05710076
  0.04184418 -0.11790338 -0.02931275 -0.26274936 -0.27398866  0.22456794
 -0.38145276 -0.06046199 -0.22898274  0.23265785] 

e_sigma:  [ 0.5823279  -0.13005098  0.13005098 -0.5823279   0.05877384 -0.01312593
  0.01312593 -0.05877384 -0.05877384  0.01312593 -0.01312593  0.05877384
 -0.5823279   0.13005098 -0.13005098  0.5823279 ] 

Cost:  10.313508796169437 

theta_1 0.7824700397841017
theta_2 0.6390021029065408
alpha_3 0.15908750659815923
alpha_4 0.25057785689812234
theta_5 0.9999999982331889
